# Чтение данных

In [ ]:
from pyspark.sql import SparkSession

# тестим что оно вообще работает

spark = SparkSession.builder \
    .appName("Lab2_ETL") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.27") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://host.docker.internal:5432/my_database_l1"
connection_properties = {
    "user": "postgres",
    "password": "pass",
    "driver": "org.postgresql.Driver"
}

df = spark.read.jdbc(url=jdbc_url, table="mock_data", properties=connection_properties)

df.show(5)

df.count()


+-------------------+------------------+------------+--------------------+----------------+--------------------+-----------------+-----------------+------------------+-----------------+----------------+--------------------+--------------+------------------+------------+----------------+--------------------+----------------+----------+----------------+--------------+---------------+-------------+--------------------+----------+--------------+--------------------+-----------+-------------+------------+--------------------+------------+--------------------+-------------+------------+-------------+----------------+--------------------+--------------------+---------------+--------------------+-------------------+-------------+----------------+--------------------+--------------+----------------+--------------------+----------------+---+
|customer_first_name|customer_last_name|customer_age|      customer_email|customer_country|customer_postal_code|customer_pet_type|customer_pet_name|customer

10000

# Преобразование данных в звезду

In [108]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

dim_product = df.select(
    df.sale_product_id.alias("product_id"),
    df.product_name.alias("name"),
    df.product_category.alias("category"),
    df.product_brand.alias("brand"),
    df.product_color.alias("color"),
    df.product_size.alias("size"),
    df.product_material.alias("material"),
    df.product_price.alias("price"),
    df.product_quantity.alias("quantity"),
    df.product_weight.alias("weight"),
    df.product_description.alias("description"),
    df.product_rating.alias("rating"),
    df.product_reviews.alias("reviews"),
    df.product_release_date.alias("release_date"),
    df.product_expiry_date.alias("expiry_date"),
    df.pet_category.alias("pet_category")
).distinct()
dim_product.show(5)

dim_product.count()


+----------+---------+--------+--------+----------+------+--------+--------------------+--------+--------------------+--------------------+--------------------+-------+------------+-----------+------------+
|product_id|     name|category|   brand|     color|  size|material|               price|quantity|              weight|         description|              rating|reviews|release_date|expiry_date|pet_category|
+----------+---------+--------+--------+----------+------+--------+--------------------+--------+--------------------+--------------------+--------------------+-------+------------+-----------+------------+
|      2737|Bird Cage|    Cage|   Voomm|Aquamarine|Medium|Aluminum|65.27000000000000...|      87|21.00000000000000...|Duis bibendum. Mo...|2.900000000000000000|    127|   5/12/2012| 10/14/2028|        Fish|
|      5634| Dog Food|    Food|Zoombeat|    Indigo| Large| Plastic|71.64000000000000...|       4|14.90000000000000...|In hac habitasse ...|3.200000000000000000|    605|    

10000

In [109]:
dim_customer = df.select(
    df.sale_customer_id.alias("customer_id"),
    df.customer_first_name.alias("first_name"),
    df.customer_last_name.alias("last_name"),
    df.customer_age.alias("age"),
    df.customer_email.alias("email"),
    df.customer_country.alias("country"),
    df.customer_postal_code.alias("postal_code"),
    df.customer_pet_name.alias("pet_name"),
    df.customer_pet_type.alias("pet_type"),
    df.customer_pet_breed.alias("pet_breed")
).distinct()
dim_customer.show(5)

dim_customer.count()

+-----------+----------+---------+---+--------------------+-----------+-----------+---------+--------+------------------+
|customer_id|first_name|last_name|age|               email|    country|postal_code| pet_name|pet_type|         pet_breed|
+-----------+----------+---------+---+--------------------+-----------+-----------+---------+--------+------------------+
|        542|   Alaster| Allright| 73|   shalltey@hibu.com|Philippines|       6227|   Kellie|     dog|Labrador Retriever|
|        716| Cassandre|  Faragan| 67|ptregonajo@comcas...|      Japan|   444-0701|   Victor|     cat|Labrador Retriever|
|        942|    Ailsun|   Budgen| 32|    hoq0@squidoo.com|      Japan|   362-0025|     Geri|     dog|          Parakeet|
|        993|   Gunilla| Trytsman| 77|mpieraccire@print...|   Thailand|      44150|   Ruthie|     cat|          Parakeet|
|       1228|      Mira|   Yukhov| 22|bpendry67@over-bl...|Philippines|       2701|Bernadine|     cat|          Parakeet|
+-----------+----------+

10000

In [110]:
dim_seller = df.select(
    df.sale_seller_id.alias("seller_id"),
    df.seller_first_name.alias("first_name"),
    df.seller_last_name.alias("last_name"),
    df.seller_email.alias("email"),
    df.seller_country.alias("country"),
    df.seller_postal_code.alias("postal_code")
).distinct()
dim_seller.show(5)

dim_seller.count()


+---------+----------+---------+--------------------+--------+-------------+
|seller_id|first_name|last_name|               email| country|  postal_code|
+---------+----------+---------+--------------------+--------+-------------+
|      222|     Griff|   Turfin|gturfin63@oaic.go...|Pakistan|        73210|
|      468|    Gillan|     Ends|gendscw@bloglovin...|  Poland|       36-007|
|      878|    Jeanne|  Shewery|jsheweryo8@unblog.fr|  France|06173 CEDEX 2|
|     1310|    Scotty|Bonallack|sbonallack8f@bizj...|  Poland|       14-230|
|     1834|  Harmonie|    Morad|    hmoradmu@cnn.com|  France|38009 CEDEX 1|
+---------+----------+---------+--------------------+--------+-------------+
only showing top 5 rows



10000

In [111]:
dim_store = df.select(
    df.store_name.alias("name"),
    df.store_location.alias("location"),
    df.store_city.alias("city"),
    df.store_state.alias("state"),
    df.store_country.alias("country"),
    df.store_phone.alias("phone"),
    df.store_email.alias("email")
).distinct().withColumn("store_id", F.monotonically_increasing_id())
dim_store.show(5)

dim_store.count()


+---------+------------+-------------+-----+----------+------------+--------------------+--------+
|     name|    location|         city|state|   country|       phone|               email|store_id|
+---------+------------+-------------+-----+----------+------------+--------------------+--------+
|     Lazz|PO Box 25788|   Las Palmas|  OAX|    Sweden|696-560-4319|smacavddy6g@frien...|       0|
|   Talane|PO Box 68620|     Chartres|   A3|  Honduras|472-917-6932|lgiacobbo5r@scien...|       1|
|Bubblebox|    Suite 58|Saint-Quentin|   B6|Kazakhstan|863-901-5165|     vturk6v@com.com|       2|
|    Voomm|   Room 1145|     Joliette|   QC|    Canada|604-183-5169|krivalants@cisco.com|       3|
|   Rhynyx|PO Box 66262|    Ardazubre|   06| Indonesia|218-133-8890|dbreawoodgp@githu...|       4|
+---------+------------+-------------+-----+----------+------------+--------------------+--------+
only showing top 5 rows



10000

In [112]:
dim_supplier = df.select(
    df.supplier_name.alias("name"),
    df.supplier_contact.alias("contact"),
    df.supplier_email.alias("email"),
    df.supplier_phone.alias("phone"),
    df.supplier_address.alias("address"),
    df.supplier_city.alias("city"),
    df.supplier_country.alias("country")
).distinct().withColumn("supplier_id", F.monotonically_increasing_id())
dim_supplier.show(5)

dim_supplier.count()


+---------+---------------+--------------------+------------+------------+-----------+---------+-----------+
|     name|        contact|               email|       phone|     address|       city|  country|supplier_id|
+---------+---------------+--------------------+------------+------------+-----------+---------+-----------+
|     Ooba|  Kylynn Bugler|kbuglerhc@busines...|239-994-3631|    Suite 87|      Titay|   Russia|          0|
|     Kazu|  Flemming Doni|fdonijd@ycombinat...|473-797-6078|    Suite 87|      Sofia|   Russia|          1|
|Topicware| Correna Gaynes|  cgayneskj@cnet.com|874-995-7853|PO Box 44041| Kazanskoye|  Ukraine|          2|
| Flashset|Shurlocke Court|scourtrn@sitemete...|710-850-7829|PO Box 86407|    Gungguh|    Palau|          3|
|   Vinder|        Deb Igo|    digonp@apple.com|536-443-3634|PO Box 76502|Mikhaylovsk|Argentina|          4|
+---------+---------------+--------------------+------------+------------+-----------+---------+-----------+
only showing top 5 

10000

In [113]:
from pyspark.sql.functions import to_date

dim_product = dim_product.withColumn("release_date", to_date("release_date", "M/d/yyyy"))
dim_product = dim_product.withColumn("expiry_date", to_date("expiry_date", "M/d/yyyy"))


In [114]:
fact_sales = df.alias("fact") \
    .join(
        dim_customer.alias("customer"),
        F.col("fact.sale_customer_id") == F.col("customer.customer_id"),
        "left"
    ) \
    .join(
        dim_seller.alias("seller"),
        F.col("fact.sale_seller_id") == F.col("seller.seller_id"),
        "left"
    ) \
    .join(
        dim_product.alias("product"),
        F.col("fact.sale_product_id") == F.col("product.product_id"),
        "left"
    ) \
    .join(
        dim_store.alias("store"),
        (F.col("fact.store_name") == F.col("store.name")) &
        (F.col("fact.store_location") == F.col("store.location")) &
        (F.col("fact.store_city") == F.col("store.city")) &
        (F.col("fact.store_state") == F.col("store.state")) &
        (F.col("fact.store_country") == F.col("store.country")) &
        (F.col("fact.store_phone") == F.col("store.phone")) &
        (F.col("fact.store_email") == F.col("store.email")),
        "left"
    ) \
    .join(
        dim_supplier.alias("supplier"),
        (F.col("fact.supplier_name") == F.col("supplier.name")) &
        (F.col("fact.supplier_contact") == F.col("supplier.contact")) &
        (F.col("fact.supplier_email") == F.col("supplier.email")) &
        (F.col("fact.supplier_phone") == F.col("supplier.phone")) &
        (F.col("fact.supplier_address") == F.col("supplier.address")) &
        (F.col("fact.supplier_city") == F.col("supplier.city")) &
        (F.col("fact.supplier_country") == F.col("supplier.country")),
        "left"
    ) \
    .select(
        F.col("fact.id").alias("sale_id"),
        F.col("customer.customer_id"),
        F.col("seller.seller_id"),
        F.col("product.product_id"),
        F.col("supplier.supplier_id"),
        F.col("store.store_id"),
        F.to_date(F.col("fact.sale_date"), "M/d/yyyy").alias("sale_date"),
        F.col("fact.sale_quantity").alias("quantity"),
        F.col("fact.sale_total_price").alias("total_price")
    )

fact_sales.show(5)

fact_sales.count()

+-------+-----------+---------+----------+-----------+--------+----------+--------+--------------------+
|sale_id|customer_id|seller_id|product_id|supplier_id|store_id| sale_date|quantity|         total_price|
+-------+-----------+---------+----------+-----------+--------+----------+--------+--------------------+
|    178|        178|      178|       178|       5496|    NULL|2021-04-29|       7|269.7300000000000...|
|    166|        166|      166|       166|       4495|    NULL|2021-10-31|       2|390.8300000000000...|
|    394|        394|      394|       394|       3296|    NULL|2021-08-31|       3|371.5100000000000...|
|    240|        240|      240|       240|       9161|    1089|2021-08-09|       7|291.5200000000000...|
|    351|        351|      351|       351|       3196|    NULL|2021-09-14|       7|180.1400000000000...|
+-------+-----------+---------+----------+-----------+--------+----------+--------+--------------------+
only showing top 5 rows



10000

# Загрузка данных в бд

CREATE TABLE dim_product (
    product_id INTEGER PRIMARY KEY,
    name VARCHAR(100),
    category VARCHAR(50),
    brand VARCHAR(50),
    color VARCHAR(50),
    size VARCHAR(50),
    material VARCHAR(50),
    price NUMERIC,
    quantity INTEGER,
    weight NUMERIC,
    description TEXT,
    rating NUMERIC,
    reviews TEXT,
    release_date DATE,
    expiry_date DATE,
    pet_category VARCHAR(50)
);

CREATE TABLE dim_customer (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INTEGER,
    email VARCHAR(100),
    country VARCHAR(50),
    postal_code VARCHAR(20),
    pet_name VARCHAR(50),
    pet_type VARCHAR(50),
    pet_breed VARCHAR(50)
);

CREATE TABLE dim_seller (
    seller_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    country VARCHAR(50),
    postal_code VARCHAR(20)
);

CREATE TABLE dim_store (
    store_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    location VARCHAR(100),
    city VARCHAR(50),
    state VARCHAR(50),
    country VARCHAR(50),
    phone VARCHAR(50),
    email VARCHAR(100)
);

CREATE TABLE dim_supplier (
    supplier_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    contact VARCHAR(100),
    email VARCHAR(100),
    phone VARCHAR(50),
    address VARCHAR(100),
    city VARCHAR(50),
    country VARCHAR(50)
);

CREATE TABLE fact_sales (
    sale_id INTEGER PRIMARY KEY,
    product_id INTEGER REFERENCES dim_product(product_id),
    customer_id INTEGER REFERENCES dim_customer(customer_id),
    seller_id INTEGER REFERENCES dim_seller(seller_id),
    store_id INTEGER REFERENCES dim_store(store_id),
    supplier_id INTEGER REFERENCES dim_supplier(supplier_id),
    sale_date DATE,
    quantity INTEGER,
    total_price NUMERIC
);


In [115]:
tables = [
    ("dim_customer", dim_customer),
    ("dim_seller", dim_seller),
    ("dim_product", dim_product),
    ("dim_store", dim_store),
    ("dim_supplier", dim_supplier),
    ("fact_sales", fact_sales)
]

for name, df in tables:
    df.write.jdbc(
        url=jdbc_url,
        table=name,
        mode="append",
        properties=connection_properties
    )

# Работа с ClickHouse

In [ ]:
clickhouse_url = "jdbc:clickhouse://clickhouse:8123/clickhouse"

clickhouse_props = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver",
    "user": "clickhouse",
    "password": "pass"
}

### Проверяем корректность подключения

In [117]:
df = spark.read \
    .format("jdbc") \
    .option("url", clickhouse_url) \
    .option("driver", clickhouse_props["driver"]) \
    .option("dbtable", "(SELECT 1 AS test_value) AS test") \
    .option("user", clickhouse_props["user"]) \
    .option("password", clickhouse_props["password"]) \
    .load()

df.show()


+----------+
|test_value|
+----------+
|         1|
+----------+



# Витрина продаж по продуктам. Цель: Анализ выручки, количества продаж и популярности продуктов.

## Топ-10 самых продаваемых продуктов.

In [118]:
top_products = fact_sales.groupBy("product_id") \
    .agg(
        F.sum("quantity").alias("total_quantity"),
        F.sum("total_price").alias("total_revenue")
    ) \
    .join(dim_product.select("product_id", "name", "category", "color", "size"), "product_id") \
    .orderBy(F.desc("total_quantity"), F.desc("total_revenue")) \
    .limit(10)

window_spec = Window.orderBy(F.desc("total_quantity"), F.desc("total_revenue"))
top_products = top_products.withColumn("rank", F.row_number().over(window_spec))

top_products.show(5)

top_products.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (rank)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="top_products", properties=clickhouse_props)


+----------+--------------+--------------------+---------+--------+------+------+----+
|product_id|total_quantity|       total_revenue|     name|category| color|  size|rank|
+----------+--------------+--------------------+---------+--------+------+------+----+
|      7620|            10|499.7300000000000...|  Cat Toy|     Toy|Fuscia|Medium|   1|
|      2557|            10|499.5900000000000...| Dog Food|     Toy|Yellow| Large|   2|
|       487|            10|498.9600000000000...|Bird Cage|     Toy|Indigo| Small|   3|
|      6600|            10|498.3500000000000...| Dog Food|    Food|  Puce| Small|   4|
|      8962|            10|497.2500000000000...| Dog Food|    Food|  Teal| Large|   5|
+----------+--------------+--------------------+---------+--------+------+------+----+
only showing top 5 rows



## Общая выручка по категориям продуктов.

In [119]:
category_sales = fact_sales.join(dim_product, "product_id") \
    .select("product_id", "category", fact_sales["total_price"]) \
    .groupBy("category") \
    .agg(F.sum("total_price").alias("total_revenue")) \
    .orderBy(F.desc("total_revenue"))

category_sales.show(5)

category_sales.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (category)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="total_revenue_by_category", properties=clickhouse_props)


+--------+--------------------+
|category|       total_revenue|
+--------+--------------------+
|     Toy|868101.6300000000...|
|    Cage|831117.9400000000...|
|    Food|830632.5500000000...|
+--------+--------------------+



## Средний рейтинг и количество отзывов для каждого продукта.

In [120]:
product_rating_stats = dim_product.select("name", "rating", "reviews") \
    .groupBy("name") \
    .agg(
        F.round(F.avg("rating"), 2).alias("avg_rating"),
        F.sum("reviews").alias("total_reviews")
    ) \
    .orderBy(F.desc("avg_rating"))

product_rating_stats.show(5)

product_rating_stats.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (avg_rating)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="product_rating_stats", properties=clickhouse_props)


+---------+----------+-------------+
|     name|avg_rating|total_reviews|
+---------+----------+-------------+
| Dog Food|      3.02|      1653413|
|  Cat Toy|      3.01|      1676222|
|Bird Cage|      3.00|      1682260|
+---------+----------+-------------+



# Витрина продаж по клиентам. Цель: Анализ покупательского поведения и сегментация клиентов.

## Топ-10 клиентов с наибольшей общей суммой покупок.

In [121]:
from pyspark.sql.functions import concat_ws

top_customers = fact_sales.groupBy("customer_id") \
    .agg(F.sum("total_price").alias("total_spent")) \
    .join(
        dim_customer.select(
            "customer_id",
            concat_ws(" ", "first_name", "last_name").alias("full_name")
        ),
        on="customer_id"
    ) \
    .select("customer_id", "full_name", "total_spent") \
    .orderBy(F.desc("total_spent")) \
    .limit(10)

top_customers.show(5)

top_customers.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (customer_id)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="top_customers", properties=clickhouse_props)


+-----------+-----------------+--------------------+
|customer_id|        full_name|         total_spent|
+-----------+-----------------+--------------------+
|       3198|    Gus Hartshorn|499.8500000000000...|
|       5426|     Hayes McKain|499.8000000000000...|
|       4928|      Dawna Impey|499.7600000000000...|
|       5364|        Ava Lomas|499.7600000000000...|
|       7620|Lavinia Horsburgh|499.7300000000000...|
+-----------+-----------------+--------------------+
only showing top 5 rows



## Распределение клиентов по странам.

In [122]:
customers_by_country = dim_customer.groupBy("country").count()

customers_by_country.show(5)

customers_by_country.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (country)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="customers_by_country", properties=clickhouse_props)

+-------------------+-----+
|            country|count|
+-------------------+-----+
|               Chad|    5|
|             Russia|  628|
|           Paraguay|   18|
|U.S. Virgin Islands|    1|
|              Yemen|   39|
+-------------------+-----+
only showing top 5 rows



## Средний чек для каждого клиента.

In [123]:
avg_receipt = fact_sales.groupBy("customer_id") \
    .agg(F.avg("total_price").alias("avg_receipt")) \
    .join(
        dim_customer.select("customer_id", "first_name", "last_name"),
        on="customer_id",
        how="left"
    ) \
    .withColumn("full_name", F.concat_ws(" ", F.col("first_name"), F.col("last_name"))) \
    .select("customer_id", "full_name", "avg_receipt")

avg_receipt.show(5)

avg_receipt.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (customer_id)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="avg_receipt_by_customer", properties=clickhouse_props)

+-----------+----------------+--------------------+
|customer_id|       full_name|         avg_receipt|
+-----------+----------------+--------------------+
|        148| Leann Ferraresi|127.3100000000000...|
|        463|     Kenton Rigg|445.9700000000000...|
|        471|  Garrick Corbyn|363.0200000000000...|
|        496|    Gregory Figg|460.4000000000000...|
|        833|Aymer Matthesius|47.51000000000000...|
+-----------+----------------+--------------------+
only showing top 5 rows



# Витрина продаж по времени. Цель: Анализ сезонности и трендов продаж.

## Месячные и годовые тренды продаж. 

In [124]:
time_sales = fact_sales.withColumn("year", F.year("sale_date")) \
                       .withColumn("month", F.month("sale_date"))

monthly_revenue = time_sales.groupBy("year", "month") \
    .agg(
        F.sum("total_price").alias("monthly_revenue"),
        F.count("*").alias("monthly_sales_count")
    )

yearly_stats = time_sales.groupBy("year") \
    .agg(
        F.sum("total_price").alias("yearly_revenue"),
        F.count("*").alias("yearly_sales_count")
    )

combined = monthly_revenue.join(yearly_stats, on="year", how="left") \
    .orderBy("year", "month")

combined.show(5)

combined.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (year, month)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="sales_time_trends", properties=clickhouse_props)

+----+-----+--------------------+-------------------+--------------------+------------------+
|year|month|     monthly_revenue|monthly_sales_count|      yearly_revenue|yearly_sales_count|
+----+-----+--------------------+-------------------+--------------------+------------------+
|2021|    1|224158.5400000000...|                874|2529852.120000000...|             10000|
|2021|    2|192348.3100000000...|                739|2529852.120000000...|             10000|
|2021|    3|207282.2000000000...|                843|2529852.120000000...|             10000|
|2021|    4|206592.8200000000...|                837|2529852.120000000...|             10000|
|2021|    5|211764.8600000000...|                828|2529852.120000000...|             10000|
+----+-----+--------------------+-------------------+--------------------+------------------+
only showing top 5 rows



## Сравнение выручки за разные периоды. 

In [125]:
time_sales_clean = fact_sales.withColumn("year", F.year("sale_date")) \
                              .withColumn("month", F.month("sale_date"))

monthly_revenue_clean = time_sales_clean.groupBy("year", "month") \
    .agg(F.sum("total_price").alias("monthly_revenue"))

yearly_revenue_clean = time_sales_clean.groupBy("year") \
    .agg(F.sum("total_price").alias("yearly_revenue"))

monthly_yearly_sales_clean = monthly_revenue_clean.join(
    yearly_revenue_clean, on="year", how="left"
).orderBy("year", "month")

monthly_yearly_sales_clean.show(5)

monthly_yearly_sales_clean.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (year, month)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="monthly_yearly_sales", properties=clickhouse_props)

monthly_avg_items_per_order.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (year, month)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="monthly_avg_items_per_order", properties=clickhouse_props)



+----+-----+--------------------+--------------------+
|year|month|     monthly_revenue|      yearly_revenue|
+----+-----+--------------------+--------------------+
|2021|    1|224158.5400000000...|2529852.120000000...|
|2021|    2|192348.3100000000...|2529852.120000000...|
|2021|    3|207282.2000000000...|2529852.120000000...|
|2021|    4|206592.8200000000...|2529852.120000000...|
|2021|    5|211764.8600000000...|2529852.120000000...|
+----+-----+--------------------+--------------------+
only showing top 5 rows



## Средний размер заказа по месяцам.

In [126]:
time_sales_items = fact_sales.withColumn("year", F.year("sale_date")) \
                             .withColumn("month", F.month("sale_date"))

monthly_items_stats = time_sales_items.groupBy("year", "month").agg(
    F.sum("quantity").alias("total_items_sold"),
    F.count("*").alias("order_count")
)

monthly_avg_items_per_order = monthly_items_stats.withColumn(
    "avg_items_per_order",
    F.col("total_items_sold") / F.col("order_count")
).select("year", "month", "avg_items_per_order") \
 .orderBy("year", "month")

monthly_avg_items_per_order.show(5)

monthly_avg_items_per_order.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (year, month)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="monthly_avg_items_per_order", properties=clickhouse_props)


+----+-----+-------------------+
|year|month|avg_items_per_order|
+----+-----+-------------------+
|2021|    1|  5.556064073226545|
|2021|    2|   5.50744248985115|
|2021|    3| 5.4104389086595495|
|2021|    4| 5.4528076463560335|
|2021|    5|  5.375603864734299|
+----+-----+-------------------+
only showing top 5 rows



# Витрина продаж по магазинам. Цель: Анализ эффективности магазинов.

## Топ-5 магазинов с наибольшей выручкой.

In [127]:
top_5_stores = fact_sales.join(dim_store, on="store_id", how="inner") \
    .groupBy("store_id", "name") \
    .agg(F.sum("total_price").alias("total_revenue")) \
    .orderBy(F.desc("total_revenue")) \
    .limit(5)

top_5_stores.show(5)

top_5_stores.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (total_revenue)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="top_5_stores", properties=clickhouse_props)

+--------+-----------+--------------------+
|store_id|       name|       total_revenue|
+--------+-----------+--------------------+
|     808|Thoughtblab|499.8000000000000...|
|    1596|      Kazio|499.6200000000000...|
|    1197|    Gabcube|499.2100000000000...|
|    1517|   Innotype|499.2000000000000...|
|     648|    Gabtype|498.8600000000000...|
+--------+-----------+--------------------+



## Распределение продаж по городам и странам.

In [128]:
store_sales_by_location = fact_sales.join(dim_store, on="store_id", how="inner") \
    .groupBy("country", "city") \
    .agg(F.sum("total_price").alias("total_revenue"))

window_spec = Window.partitionBy("country")

store_sales_by_location = store_sales_by_location \
    .withColumn("total_revenue_by_country", F.sum("total_revenue").over(window_spec)) \
    .orderBy(F.desc("total_revenue"))

store_sales_by_location.show(5)

store_sales_by_location.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (country, city)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="store_sales_by_location", properties=clickhouse_props)

+---------+----------+--------------------+------------------------+
|  country|      city|       total_revenue|total_revenue_by_country|
+---------+----------+--------------------+------------------------+
|    China| Stockholm|2056.830000000000...|    79995.69000000000...|
|    China| København|986.4600000000000...|    79995.69000000000...|
|    China|  Västerås|956.2900000000000...|    79995.69000000000...|
|    China|     Luleå|823.2300000000000...|    79995.69000000000...|
|Indonesia|Washington|799.8300000000000...|    36486.61000000000...|
+---------+----------+--------------------+------------------------+
only showing top 5 rows



## Средний чек для каждого магазина.

In [129]:
average_check_by_store = fact_sales.groupBy("store_id") \
    .agg(
        (F.sum("total_price") / F.count("*")).alias("average_check")
    ) \
    .join(dim_store.select("store_id", "name"), on="store_id", how="inner") \
    .select("store_id", "name", "average_check")

average_check_by_store.show(5)

average_check_by_store.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (average_check)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="average_check_by_store", properties=clickhouse_props)


+--------+-----------+--------------------+
|store_id|       name|       average_check|
+--------+-----------+--------------------+
|     964|     Agimba|192.3500000000000...|
|      29|    Cogibox|110.3600000000000...|
|      26|Topiclounge|447.9700000000000...|
|     474|       Omba|29.57000000000000...|
|    1010|       Oozz|278.7800000000000...|
+--------+-----------+--------------------+
only showing top 5 rows



# Витрина продаж по поставщикам. Цель: Анализ эффективности поставщиков.

## Топ-5 поставщиков с наибольшей выручкой.

In [130]:
dim_supplier_renamed = dim_supplier.withColumnRenamed("name", "supplier_name")

top_5_suppliers = fact_sales.join(dim_product, on="product_id", how="inner") \
    .join(dim_supplier_renamed, on="supplier_id", how="inner") \
    .groupBy("supplier_id", "supplier_name") \
    .agg(F.sum("total_price").alias("total_revenue")) \
    .orderBy(F.desc("total_revenue")) \
    .limit(5)

top_5_suppliers.show(5)

top_5_suppliers.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (total_revenue)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="top_5_suppliers", properties=clickhouse_props)

+-----------+-------------+--------------------+
|supplier_id|supplier_name|       total_revenue|
+-----------+-------------+--------------------+
|       2490|   Brainverse|499.8500000000000...|
|       6258|        Jamia|499.8000000000000...|
|       7286|        Eabox|499.7600000000000...|
|       4222|      Demimbu|499.7600000000000...|
|       3919|   Browsezoom|499.7300000000000...|
+-----------+-------------+--------------------+



## Средняя цена товаров от каждого поставщика. 

In [131]:
avg_price_per_supplier = fact_sales.alias("f") \
    .join(dim_product.alias("p"), F.col("f.product_id") == F.col("p.product_id")) \
    .join(dim_supplier.alias("s"), F.col("f.supplier_id") == F.col("s.supplier_id")) \
    .groupBy(F.col("s.supplier_id"), F.col("s.name")) \
    .agg(F.avg(F.col("p.price")).alias("avg_price"))

avg_price_per_supplier.show(5)

avg_price_per_supplier.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (avg_price)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="avg_price_per_supplier", properties=clickhouse_props)

+-----------+----------+--------------------+
|supplier_id|      name|           avg_price|
+-----------+----------+--------------------+
|       2851|   Cogidoo|80.87000000000000...|
|       6229|Jabbertype|75.87000000000000...|
|       6863|      Kazu|18.67000000000000...|
|       5941|    Camido|83.25000000000000...|
|       4624| Babbleset|8.450000000000000...|
+-----------+----------+--------------------+
only showing top 5 rows



## Распределение продаж по странам поставщиков.

In [132]:
sales_by_supplier_country = fact_sales \
    .join(dim_supplier, "supplier_id") \
    .groupBy("country") \
    .agg(F.sum("total_price").alias("total_revenue"))

sales_by_supplier_country.show(5)

sales_by_supplier_country.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (country)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="sales_by_supplier_country", properties=clickhouse_props)

+--------+--------------------+
| country|       total_revenue|
+--------+--------------------+
|    Chad|2051.030000000000...|
|  Russia|149206.7500000000...|
|Paraguay|2957.860000000000...|
|   Yemen|11414.63000000000...|
| Senegal|3162.410000000000...|
+--------+--------------------+
only showing top 5 rows



# Витрина качества продукции. Цель: Анализ отзывов и рейтингов товаров.

## Продукты с наивысшим и наименьшим рейтингом.

In [133]:
products_ordered_by_rating = dim_product \
    .select("product_id", "name", "rating") \
    .orderBy(F.desc("rating"))

products_ordered_by_rating.show(5)

products_ordered_by_rating.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (rating)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="products_ordered_by_rating", properties=clickhouse_props)

+----------+---------+--------------------+
|product_id|     name|              rating|
+----------+---------+--------------------+
|      2332|Bird Cage|5.000000000000000000|
|      4203|  Cat Toy|5.000000000000000000|
|      9186| Dog Food|5.000000000000000000|
|      1783|  Cat Toy|5.000000000000000000|
|      8070|  Cat Toy|5.000000000000000000|
+----------+---------+--------------------+
only showing top 5 rows



## Корреляция между рейтингом и объемом продаж. 

In [134]:
rating_sales_corr = fact_sales.join(dim_product, "product_id") \
    .groupBy("product_id", "rating") \
    .agg(F.sum(fact_sales["quantity"]).alias("total_quantity"))

rating_sales_corr.show(5)

rating_sales_corr.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (rating)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="rating_sales_corr", properties=clickhouse_props)

+----------+--------------------+--------------+
|product_id|              rating|total_quantity|
+----------+--------------------+--------------+
|       148|2.300000000000000000|             5|
|       463|2.800000000000000000|             6|
|       471|3.600000000000000000|             8|
|       496|2.000000000000000000|             4|
|       833|3.300000000000000000|             2|
+----------+--------------------+--------------+
only showing top 5 rows



## Продукты с наибольшим количеством отзывов.

In [135]:
top_reviewed_products = dim_product.select("product_id", "name", "reviews") \
    .orderBy(F.desc("reviews"))

top_reviewed_products.show(5)

top_reviewed_products.write \
    .option("createTableOptions", "ENGINE = MergeTree() ORDER BY (product_id)") \
    .mode("overwrite") \
    .jdbc(url=clickhouse_url, table="top_reviewed_products", properties=clickhouse_props)

+----------+---------+-------+
|product_id|     name|reviews|
+----------+---------+-------+
|      1938|  Cat Toy|   1000|
|      1073|Bird Cage|   1000|
|      9735|Bird Cage|   1000|
|      8992|Bird Cage|   1000|
|      6383|  Cat Toy|   1000|
+----------+---------+-------+
only showing top 5 rows

